In [1]:
#create folder in batchs
import os, sys
path1 = 'D:\VGGface2\overlap_vggface1_2/'  #指定名称文件夹所在路径
path2 = 'D:/cnnface/female_male_test_51_addnoise/noise_0.05/'    #新建文件夹所在路径
 
def MkDir():
    dirs = os.listdir(path1)
    for dir in dirs:
        file_name = path2 + str(dir)
        os.mkdir(file_name)
        
MkDir()

In [ ]:
#extract dnn_activation
import argparse
from torchvision import transforms
from torch.utils.data import DataLoader
from dnnbrain.dnn import analyzer
from dnnbrain.dnn import io as dnn_io

stim = 'D:\cnnface\Identity_analysis/identity2_train.csv'
netloader = dnn_io.NetLoader('vggface')
imgcropsize = netloader.img_size
transform = transforms.Compose([transforms.Resize(imgcropsize),
                                    transforms.ToTensor()])  
picdataset = dnn_io.PicDataset(stim, transform=transform)
picdataloader = DataLoader(picdataset, batch_size=8, shuffle=False)
dnn_act = analyzer.dnn_activation(picdataloader, 'vggface', 'fc3')


In [3]:
#Get classification result of network.
import numpy as np
import torch
from torchvision import transforms

from dnnbrain.dnn.models import dnn_test_model
from dnnbrain.dnn.io import PicDataset,NetLoader,DataLoader
from cnnface.core.vgg_identity_recons import Vgg_identity

vggid = Vgg_identity()
vggid.load_state_dict(torch.load('F:/Code/pretrained_model/vgg_identity2_CrossEntro.pth'))

imgcsv_path =  'D:\cnnface\Identity_analysis/bf_noise5000.csv'
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor()])
PicSet = PicDataset(imgcsv_path,transform)
Picloader = DataLoader(PicSet,batch_size=32)

output = dnn_test_model(Picloader,vggid)

#save label to txt file
label = output[0]
label_f = np.argwhere(label==0)
label_m = np.argwhere(label==1)

print(label[:10])

Now loading batch 1
Now loading batch 2
Now loading batch 3
Now loading batch 4
Now loading batch 5
Now loading batch 6
Now loading batch 7
Now loading batch 8
Now loading batch 9
Now loading batch 10
Now loading batch 11
Now loading batch 12
Now loading batch 13
Now loading batch 14
Now loading batch 15
Now loading batch 16
Now loading batch 17
Now loading batch 18
Now loading batch 19
Now loading batch 20
Now loading batch 21
Now loading batch 22
Now loading batch 23
Now loading batch 24
Now loading batch 25
Now loading batch 26
Now loading batch 27
Now loading batch 28
Now loading batch 29
Now loading batch 30
Now loading batch 31
Now loading batch 32
Now loading batch 33
Now loading batch 34
Now loading batch 35
Now loading batch 36
Now loading batch 37
Now loading batch 38
Now loading batch 39
Now loading batch 40
Now loading batch 41
Now loading batch 42
Now loading batch 43
Now loading batch 44
Now loading batch 45
Now loading batch 46
Now loading batch 47
Now loading batch 48
N

In [4]:
print(label_f.shape)
print(label_m.shape)

np.savetxt('D:\cnnface\Identity_analysis/label_ac.txt',label_f)
np.savetxt('D:\cnnface\Identity_analysis/label_nc.txt',label_m)

(4476, 1)
(524, 1)


 
 ## Calculate the CI

In [7]:
import numpy as np
from dnnbrain.dnn import io
import os
from PIL import Image 
import sys
sys.path.append("..")
from cnnface.core.Image_processing import average_img

csv_file = 'D:\cnnface\Identity_analysis/bf_noise5000.csv'
index_f = np.loadtxt('D:\cnnface\Identity_analysis/label_ac.txt').astype('int64')
index_m = np.loadtxt('D:\cnnface\Identity_analysis/label_nc.txt').astype('int64')

savepath_aver_f = 'D:\cnnface\Identity_analysis\Classify_img/aver_ac_noise.jpg'
savepath_aver_m = 'D:\cnnface\Identity_analysis\Classify_img/aver_nc_noise.jpg'
savepath_CI = 'D:\cnnface\Identity_analysis\Classify_img/ci_noise_40.jpg'

#Get pictures path from stim_csv and label txt.
picset= io.PicDataset(csv_file)
pre_picpath = picset.picpath
sub_picpath = picset.picname

f_picpath = [os.path.join(pre_picpath,sub_picpath[i]) for i in index_f]
m_picpath = [os.path.join(pre_picpath,sub_picpath[i]) for i in index_m]

#average all of the pictures as their label.
aver_img_f = average_img(f_picpath)
aver_img_m = average_img(m_picpath)

#calculate the CI and superimpose image and subtract image.
ci = aver_img_f.astype('int64') - aver_img_m.astype('int64')
ci = ci - ci.min()
ci = ci * 5
print(ci.max(),ci.min())

(Image.fromarray(aver_img_f.astype('uint8'))).save(savepath_aver_f,quality=95)
(Image.fromarray(aver_img_m.astype('uint8'))).save(savepath_aver_m,quality=95)
(Image.fromarray(ci.astype('uint8'))).save(savepath_CI,quality=95)

60 0


In [10]:
savepath_CI = 'D:\cnnface\Identity_analysis\Classify_img/ci_noise_255.jpg'
nor = lambda x:(x-x.min())/(x.max() - x.min()) 
ci  = nor(ci) * 255
(Image.fromarray(ci.astype('uint8'))).save(savepath_CI,quality=95)

In [24]:
base_face = np.array(Image.open('D:\cnnface\Identity_analysis/face_template/identity2_morph.jpg'))

ci =  np.array(Image.open('D:\cnnface\Identity_analysis/Classify_img/ci_noise.jpg'))

bf_add = base_face + ci 
bf_sub = base_face - ci 

bf_add[bf_add>255] = 255
bf_sub[bf_sub<0] = 0

nor = lambda x:(x-x.min())/(x.max() - x.min()) 
bf_add_x = nor(bf_add) *255
bf_sub_x = nor(bf_sub) *255
(Image.fromarray(bf_add_x.astype('uint8'))).save('D:\cnnface\Identity_analysis/Classify_img/bf_add_120.jpg',quality=95)
(Image.fromarray(bf_sub_x.astype('uint8'))).save('D:\cnnface\Identity_analysis/Classify_img/bf_sub_120.jpg',quality=95)


In [ ]:
import numpy as np
from PIL import Image

usecols = list(range(513))
usecols.remove(0)
sinusoid = np.loadtxt('D:/cnnface/rcrci/sinusoid/cycle/sinuoid_4.csv',delimiter=',',skiprows=1,usecols=usecols)
sinusoid = ((sinusoid - sinusoid.min())/(sinusoid.max() - sinusoid.min()))*255
img_sinu = Image.fromarray(sinusoid)
img_sinu.show()

In [ ]:
#generateSinusoid
from PIL import Image
import numpy as np

img_size = 512/2
cycles = 2
angle = [0,30,60,90,120,150]
phase = [0,np.pi/2]

def generateSingleSinusoid(img_size, cycles, angle, phase):
    angle = np.radians(angle)
    sinepath = (np.linspace(0,cycles,img_size))[:,np.newaxis].repeat(img_size,axis=1)
    sinusoid = (sinepath*np.sin(angle) + sinepath.T*np.cos(angle)) * 2 *np.pi
    sinusoid = np.sin(sinusoid + phase)
    return sinusoid


def generateCombinSinusoid(img_size,cycles,angle,phase):
    sinusoid_combined = []
    for p in phase:
        for a in angle:
            sinusoid = generateSingleSinusoid(img_size,cycles,a,p)
            sinusoid = np.random.randn() * sinusoid
            sinusoid_combined.append(sinusoid)
    sinusoid_combined = np.array(sinusoid_combined)
    sinusoid_combined = np.sum(sinusoid_combined,axis=0)
    sinusoid_combined = (sinusoid_combined - sinusoid_combined.min()) / (sinusoid_combined.max() - sinusoid_combined.min())
    sinusoid_combined = sinusoid_combined *255
    return sinusoid_combined


scales = [(2**i)**2 for i in range(5)]
img_size = [512,256,128,64,32]
sinusoid_1 = generateCombinSinusoid(img_size[0],cycles,angle,phase)
sinusoid_2 = [generateCombinSinusoid(img_size[1],cycles,angle,phase) for i in range(4)]
sinusoid_3 = [generateCombinSinusoid(img_size[1],cycles,angle,phase) for i in 16]
sinusoid_4 = [generateCombinSinusoid(img_size[1],cycles,angle,phase) for i in 64]
sinusoid_5 = [generateCombinSinusoid(img_size[1],cycles,angle,phase) for i in 256]

for col in range(8):
    for row in range(8):
        sinusoid_com = sinusoid_3[col] + sinusoid_3[col+1]
sinusoid_com1 = np.vstack((sinusoid_2[0],sinusoid_2[1]))
sinusoid_com2 = np.vstack((sinusoid_2[2],sinusoid_2[3]))
print(sinusoid_com1.shape)
print(sinusoid_com2.shape)
sinusoid_com = np.hstack((sinusoid_com1,sinusoid_com2))

img_sinu = Image.fromarray(sinusoid_com)
img_sinu.show()

In [ ]:
# convert picture to L

In [ ]:
import sys
sys.path.append("..")
from cnnface.core import Image_processing

CI_ori_p = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_ori.jpg'
CI_ori_5 = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_ori_5.jpg'
CI_ori_20 = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_ori_20.jpg'
CI_sinusoid_nor = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_sinusoid_nor.jpg'
CI_sinusoid5_nor = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_sinusoid5_nor.jpg'

# Image_processing.image_freq_hist_plot(CI_ori_p)
# Image_processing.image_freq_hist_plot(CI_ori_5)
# Image_processing.image_freq_hist_plot(CI_ori_20)
# Image_processing.image_freq_hist_plot(CI_sinusoid_nor)
# Image_processing.image_freq_hist_plot(CI_sinusoid5_nor)


Image_processing.img_zmap(CI_ori_p)
Image_processing.img_zmap(CI_ori_5)
Image_processing.img_zmap(CI_ori_20)
Image_processing.img_zmap(CI_sinusoid_nor)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as pg

CI_sinusoid_nor = 'D:/cnnface/female_male_test_51_addnoise\Face_template/classification_noise/CI_sinusoid_nor.jpg'
CI = Image.open(CI_sinusoid_nor)
CI_arr = np.array(CI)

ci_map = plt.imshow(CI_arr,cmap = 'jet',interpolation='nearest')
save_path  ='D:/cnnface/female_male_test_51_addnoise/Face_template/classification_noise/CI_jet.jpg'
pg.imsave(save_path,CI_arr,cmap = 'jet',format='jpg',dpi=1000)



In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import matplotlib.pyplot as plt

#read image path from csv.file
csv_file = 'D:/cnnface/female_male_test_51_addnoise/rcicr_sinusoid_noise.csv'
pd = pd.read_csv(csv_file,skiprows=1)
stimID = np.array(pd['stimID'])
with open(csv_file,'r') as f:
    picpath = f.readline().rstrip()
    
#read base img 
base_img = Image.open('D:/cnnface/female_male_test_51_addnoise/Face_template/face_template/frame054_gray_512.jpg')
baseimg_arr = np.array(base_img).astype('int32')
img_min = []
img_mean = []
img_max = []

#read image from array
#calculate the min mean max of singal pictures
for x,i in enumerate(stimID):
    picimg = Image.open(os.path.join(picpath, i))
    picimg_arr = np.array(picimg).astype('int32')
    contrast = picimg_arr - baseimg_arr
    # - base image to get the pixel (positive and negative)
    con_min = contrast.min()
    con_mean = contrast.mean()
    con_max = contrast.max()
    
    if x <5:
        plt.hist(contrast)
        plt.show()
    img_min.append(con_min)
    img_mean.append(con_mean)
    img_max.append(con_max)
#sum and plot the hist of min ,mean,max of all pictures

plt.hist(img_min)
plt.show()
plt.hist(img_mean)
plt.show()
plt.hist(img_max)
plt.show()

In [ ]:
#load RData file to python
import rpy2.robjects as robjects
import numpy as np
robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/pythonRead/sinusoid1000_patches.RData")

x = robjects.r['patches']
print(np.array(x))

In [ ]:
#convert .RData to npy file
import rpy2.robjects as robjects
import numpy as np

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_07.Rdata")
params = robjects.r['stimuli_params'][0]

params_1000 = np.array(params)

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_30.Rdata")
params = robjects.r['stimuli_params'][0]

params_2000 = np.array(params)

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_45.Rdata")
params = robjects.r['stimuli_params'][0]

params_3000 = np.array(params)

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_11_02.Rdata")
params = robjects.r['stimuli_params'][0]

params_4000 = np.array(params)

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_11_22.Rdata")
params = robjects.r['stimuli_params'][0]

params_5000 = np.array(params)

p = np.concatenate((params_1000,params_2000),axis=0)
p = np.concatenate((p,params_3000),axis=0)
p = np.concatenate((p,params_4000),axis=0)
p = np.concatenate((p,params_5000),axis=0)

np.save('D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/params_5000',p)



In [ ]:
#convert .RData to npy file
import rpy2.robjects as robjects
import numpy as np

robjects.r['load']("D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/rcic_seed_1_time_9月_18_2019_10_07.Rdata")
p = robjects.r['p']
print(p.names)
patches = p[0]
patchidx = p[1]

patches = np.array(patches)
patchidx = np.array(patchidx)

np.save('D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/patches.npy',patches)
np.save('D:/cnnface/female_male_test_51_addnoise/Face_template/meta_data/patchidx.npy',patchidx)


